In [73]:
# Importing Libraries

import numpy as np
import pandas as pd
import os

# Libraries for Plotting

import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for Data Preprocessing

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Importing Libraries for Time Series Analysis

import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import adfuller

# Libraries for Modelling


from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import statsmodels.api as sm


# Library for copying

import copy

In [18]:
# Identifying current directory

current_directory = os.getcwd()
print(current_directory)

/Users/suparnaroy/Library/CloudStorage/GoogleDrive-roysuparna.sr@gmail.com/Other computers/My Laptop/All about Manchester/My Course/Dissertation/Notebooks


In [19]:
df = pd.read_csv('Data/Prepared_Data.csv')
df.drop('Unnamed: 0', axis = 1, inplace= True) ## Dropping Column with index as column
df.head()

,Year,Pork,Beef_and_veal,Poultry,Sheep,Population
0,1990,21638.20,12415.74,2054.65,1809.26,448805.382
1,1991,21234.93,12311.40,8099.46,1753.56,450298.107
2,1992,20991.89,15833.92,9777.32,2051.71,688358.946
3,1993,24072.31,15307.84,9799.41,2070.24,705070.033
4,1994,23715.50,14543.26,10474.86,1981.30,705611.100


In [20]:
df['Year'] = pd.to_datetime(df['Year'], format = '%Y').dt.strftime('%Y') ## Changing Year to datetime type
df.set_index('Year')
df.drop('Population', axis=1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           32 non-null     object 
 1   Pork           32 non-null     float64
 2   Beef_and_veal  32 non-null     float64
 3   Poultry        32 non-null     float64
 4   Sheep          32 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.4+ KB


In [21]:
df.head()

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,1990,21638.20,12415.74,2054.65,1809.26
1,1991,21234.93,12311.40,8099.46,1753.56
2,1992,20991.89,15833.92,9777.32,2051.71
3,1993,24072.31,15307.84,9799.41,2070.24
4,1994,23715.50,14543.26,10474.86,1981.30


In [22]:
df_copy = copy.deepcopy(df)

In [23]:
# Copying the original series
original_beef_and_veal = df['Beef_and_veal'].copy()
original_poultry = df['Poultry'].copy()
original_sheep = df['Sheep'].copy()

In [24]:
# Importing Population including forecasts

Pop = pd.read_csv("Data/Population_Forecast_Holt.csv")
Pop.drop('Unnamed: 0', axis = 1, inplace= True) ## Dropping Column with index as column
Pop['Year'] = pd.to_datetime(Pop['Year'], format = '%Y').dt.strftime('%Y') ## Changing Year to datetime type
Pop.head()

,Year,Value
0,1990,448805.382
1,1991,450298.107
2,1992,688358.946
3,1993,705070.033
4,1994,705611.100


In [25]:
original_pop = copy.deepcopy(Pop)

### Data Preprocessing

In [26]:
## Population is non-stationary. Differencing to ensure data is stationary.

Pop['Pop_diff'] = Pop['Value'].diff().dropna()
Pop

,Year,Value,Pop_diff
0,1990,448805.382000,NaN
1,1991,450298.107000,1492.725000
2,1992,688358.946000,238060.839000
3,1993,705070.033000,16711.087000
4,1994,705611.100000,541.067000
5,1995,705768.653000,157.553000
6,1996,705842.588000,73.935000
7,1997,705880.858000,38.270000
8,1998,705845.172000,-35.686000
9,1999,705683.958000,-161.214000


In [27]:
Diff_Pop_df = copy.deepcopy(Pop)

In [28]:
## Beef_and_veal, Poultry, and Sheep are non-stationary. Differencing to ensure data is stationary.

df['Beef_and_veal_diff'] = df['Beef_and_veal'].diff().dropna()
df['Poultry_diff'] = df['Poultry'].diff().dropna()
df['Sheep_diff'] = df['Sheep'].diff().dropna()

In [29]:
df.drop(df.loc[df['Year'] == '1990'].index, inplace= True) # Dropping the row corresponding to year 1990 as it is NaN after differencing
Pop.drop(Pop.loc[Pop['Year'] == '1990'].index, inplace = True) # Dropping the row corresponding to year 1990 as it is NaN after differencing
Pop.drop(Pop.loc[Pop['Year'] > '2021'].index, inplace = True) # Dropping the forecasted population data from 2022

In [30]:
Pop

,Year,Value,Pop_diff
1,1991,450298.107,1492.725
2,1992,688358.946,238060.839
3,1993,705070.033,16711.087
4,1994,705611.100,541.067
5,1995,705768.653,157.553
6,1996,705842.588,73.935
7,1997,705880.858,38.270
8,1998,705845.172,-35.686
9,1999,705683.958,-161.214
10,2000,716294.554,10610.596


In [31]:
exog_Pop = Pop['Pop_diff']

In [32]:
df.drop(['Beef_and_veal', 'Poultry', 'Sheep'], axis= 1, inplace= True) # Dropping the original series as Differenced series are required
df.set_index('Year', inplace=True)
df.head()

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
Year,,,,
1991,21234.93,-104.34,6044.81,-55.70
1992,20991.89,3522.52,1677.86,298.15
1993,24072.31,-526.08,22.09,18.53
1994,23715.50,-764.58,675.45,-88.94
1995,23678.54,-928.25,539.30,-40.91


## Data Modelling: VARMAX(p=1, q=2) with Population as Exogenous Variable

In [33]:
# Splitting the data into train and test sets
n_obs = len(df)
train_size = int(0.8 * n_obs)
train, test = df[0:train_size], df[train_size:]

In [34]:
# Standardizing the data

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)

test_scaled = scaler.transform(test)

In [35]:
# Splitting exogenous variable: Population into train and test

exog_train = exog_Pop[0:train_size]
exog_test = exog_Pop[train_size:]

In [36]:
# Standardizing the exogenous variable

scaler_exog = StandardScaler()

exog_train_scaled = scaler_exog.fit_transform(np.array(exog_train).reshape(-1,1)) 
exog_test_scaled = scaler_exog.transform(np.array(exog_test).reshape(-1,1))

In [37]:
# Specifying the order (p,q) for VARMA model; p is the AR order, q is the MA order
order = (1,2)  # Such as: (p=1, q=2)

# Fitting the VARMAX model
model = VARMAX(train_scaled, order=order, exog=exog_train_scaled, enforce_stationarity=True, enforce_invertibility=True)
model_fitted = model.fit(disp=False, maxiter= 200, method='lbfgs')  # setting disp=False to suppress convergence messages

# Forecasting
forecast = model_fitted.forecast(steps=len(test), exog = exog_test_scaled)

print(forecast)


/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


[[ 0.305399   -0.39566057  0.1954074  -0.32424557]
 [ 0.14994009  0.02037127 -0.36756797  0.3387049 ]
 [ 0.24480807 -0.29608058 -0.10312953 -0.35229813]
 [ 0.29138686 -0.10881269 -0.25696769 -0.09277118]
 [ 0.32311771 -0.20269782 -0.1973974  -0.21145539]
 [ 0.3407298  -0.1664521  -0.22340308 -0.15959161]
 [ 0.3635871  -0.18730879 -0.22762514 -0.17875935]]


/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


In [38]:
print(model_fitted.mle_retvals)

{'fopt': 0.5947964884178615, 'gopt': array([-3.60959161e-01,  8.74351181e-01,  5.11914856e-01, -6.62816478e-01,
        3.78181088e-01, -2.86895988e-02,  1.71190486e-01, -6.74874406e-02,
        1.14082806e+00,  1.82607266e-03, -8.78526140e-02,  1.08074505e-01,
       -1.28580078e+00, -1.94743314e-01, -3.27414120e-01,  3.23897658e-01,
        1.03950983e-01,  2.44507707e-01,  3.77412893e-01,  1.87723104e-01,
        1.54018160e-01,  1.63252645e-01,  1.49979493e-01, -1.44257355e-02,
       -4.71343148e-02, -1.91700590e-01,  2.81476892e-01, -1.51567933e-01,
       -9.75480950e-02,  2.33000845e-01, -1.17768895e-01,  3.52708628e-01,
       -2.29714974e-02,  4.62140006e-01, -3.92078928e-01,  4.17127194e-01,
        1.40111517e-02, -1.06739425e-01,  1.53752120e-01, -2.80450241e-01,
        2.37063539e-01, -6.67266028e-01,  7.26978895e-01, -6.11391096e-01,
       -4.43686878e-01, -2.40546316e-01,  2.26735265e-01, -4.95783475e-01,
        1.54871170e-01, -1.82672351e-01,  2.09632544e-01, -2.60

In [39]:
# Convert test_scaled back to dataframe
test_scaled_df = pd.DataFrame(test_scaled, columns=df.columns)

# Convert train_scaled back to dataframe
train_scaled_df = pd.DataFrame(train_scaled, columns=df.columns)

In [40]:
# Convert forecasted values into DataFrame for easier handling
forecast_df = pd.DataFrame(forecast, index=df.index[-len(test_scaled):], columns=df.columns)
forecast_df

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
Year,,,,
2015,0.305399,-0.395661,0.195407,-0.324246
2016,0.149940,0.020371,-0.367568,0.338705
2017,0.244808,-0.296081,-0.103130,-0.352298
2018,0.291387,-0.108813,-0.256968,-0.092771
2019,0.323118,-0.202698,-0.197397,-0.211455
2020,0.340730,-0.166452,-0.223403,-0.159592
2021,0.363587,-0.187309,-0.227625,-0.178759


### Model Evaluation: VARMAX(1, 2)

In [41]:
# Calculate evaluation metrics for each series
for col in df.columns[:]:
    print(f"Results for {col}")
    print("-------------------------")
    
    mae = mean_absolute_error(test_scaled_df[col], forecast_df[col])
    rmse = mean_squared_error(test_scaled_df[col], forecast_df[col], squared=False)
    mape = mean_absolute_percentage_error(test_scaled_df[col], forecast_df[col])
    
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}")
    print("\n")

Results for Pork
-------------------------
MAE: 1.1333258922394156
RMSE: 1.1561225681670506
MAPE: 0.7909037312909479


Results for Beef_and_veal_diff
-------------------------
MAE: 0.18627938166157887
RMSE: 0.23726969539272175
MAPE: 3.2188269081066254


Results for Poultry_diff
-------------------------
MAE: 0.25619128800672897
RMSE: 0.26348911710659256
MAPE: 1.6964624890503441


Results for Sheep_diff
-------------------------
MAE: 0.3486865463217034
RMSE: 0.4965607530300537
MAPE: 1.6778487015862886




#### Retraining the VARMAX(1, 2) model using the entire dataset to forecast meat consumption of all types from 2022 through 2025

In [42]:
df.head()

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
Year,,,,
1991,21234.93,-104.34,6044.81,-55.70
1992,20991.89,3522.52,1677.86,298.15
1993,24072.31,-526.08,22.09,18.53
1994,23715.50,-764.58,675.45,-88.94
1995,23678.54,-928.25,539.30,-40.91


In [43]:
## Standardizing the data

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [44]:
# Standardizing the exogenous variable: Population

scaler_exog = StandardScaler()
exog_scaled = scaler_exog.fit_transform(np.array(exog_Pop).reshape(-1,1))

In [45]:
order = (1,2)  # Such as: (p=1, q=2)

# Fitting the VARMAX model
model = VARMAX(df_scaled, order=order, exog=exog_scaled, enforce_stationarity=True, enforce_invertibility=True)
model_fitted = model.fit(disp=False, method='lbfgs', maxiter=200)  # set disp=False to suppress convergence messages

/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [46]:
# Dropping data before 2022 to use forecasted population as exogenous variables for future forecasts

Diff_Pop_df.drop(Diff_Pop_df[Diff_Pop_df['Year'] < '2022'].index, inplace= True) 
Diff_Pop_df.head()

,Year,Value,Pop_diff
32,2022,743656.395355,473.799355
33,2023,743715.879589,59.484233
34,2024,743763.466975,47.587387
35,2025,743801.536885,38.069909


In [47]:
# Standardizing the exogenous variable: Population

exog_test_scaled = scaler_exog.transform(np.array(Diff_Pop_df['Pop_diff']).reshape(-1,1))

In [48]:
# Using forecast
future_forecasts = model_fitted.forecast(steps=4, exog = exog_test_scaled)
print(future_forecasts)

[[ 1.12524419 -0.0693386   0.29604625 -0.73114172]
 [ 1.13998005  0.63546344 -0.36724254  0.27802954]
 [ 1.15211765 -0.11964477 -0.36909688 -0.04702936]
 [ 0.98732842 -0.1208199  -0.11255959 -0.07999282]]


/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/suparnaroy/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


In [49]:
# Inverse transforming the forecasts to get original scale

forecast_rescaled = scaler.inverse_transform(future_forecasts)
forecast_rescaled

array([[ 2.66703042e+04, -1.09518767e+02,  9.22393101e+02,
        -6.83242016e+01],
       [ 2.66921617e+04,  3.95280865e+02,  2.00515310e+02,
         3.44304856e+00],
       [ 2.67101653e+04, -1.45549497e+02,  1.98497178e+02,
        -1.96735266e+01],
       [ 2.64657353e+04, -1.46391165e+02,  4.77694676e+02,
        -2.20177248e+01]])

In [50]:
# Converting rescaled forecasted data into a DataFrame

actual_forecast_df = pd.DataFrame(forecast_rescaled, columns=df.columns)
actual_forecast_df

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
0,26670.304161,-109.518767,922.393101,-68.324202
1,26692.161697,395.280865,200.515310,3.443049
2,26710.165259,-145.549497,198.497178,-19.673527
3,26465.735271,-146.391165,477.694676,-22.017725


In [51]:
# Function for Reverse Differencing

def reverse_differencing(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
        # Roll back 1st Diff
        # df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
        df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
    return df_fc

In [52]:
original_beef_and_veal.tail()

27    10810.73
28    11012.63
29    10818.63
30    10601.35
31    10560.19
Name: Beef_and_veal, dtype: float64

In [53]:
last_obs_beef_and_veal = float(original_beef_and_veal.iloc[-1:])
last_obs_poultry = float(original_poultry.iloc[-1:])
last_obs_sheep = float(original_sheep.iloc[-1:])

In [54]:
df_train = df_copy.drop(['Year', 'Pork'], axis = 1)
print(df_train.tail())

forecasted_df = actual_forecast_df.drop(['Pork'], axis = 1)
print(forecasted_df)

    Beef_and_veal   Poultry    Sheep
27       10810.73  19935.89  1338.59
28       11012.63  20477.58  1345.67
29       10818.63  20540.11  1323.69
30       10601.35  20530.71  1292.33
31       10560.19  20660.75  1303.06
   Beef_and_veal_diff  Poultry_diff  Sheep_diff
0         -109.518767    922.393101  -68.324202
1          395.280865    200.515310    3.443049
2         -145.549497    198.497178  -19.673527
3         -146.391165    477.694676  -22.017725


In [55]:
# Reverse Differencing the Data to get back the original scale

df_results = reverse_differencing(df_train, forecasted_df, second_diff = False)
print(df_results)


   Beef_and_veal_diff  Poultry_diff  Sheep_diff  Beef_and_veal       Poultry  \
0         -109.518767    922.393101  -68.324202   10450.671233  21583.143101   
1          395.280865    200.515310    3.443049   10845.952097  21783.658412   
2         -145.549497    198.497178  -19.673527   10700.402600  21982.155589   
3         -146.391165    477.694676  -22.017725   10554.011435  22459.850265   

         Sheep  
0  1234.735798  
1  1238.178847  
2  1218.505320  
3  1196.487596  


In [56]:
actual_forecast_df

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
0,26670.304161,-109.518767,922.393101,-68.324202
1,26692.161697,395.280865,200.515310,3.443049
2,26710.165259,-145.549497,198.497178,-19.673527
3,26465.735271,-146.391165,477.694676,-22.017725


In [57]:
results = pd.concat([actual_forecast_df, df_results], axis = 1) # Concatenating to get all forecasted series

In [59]:
# Dropping differenced columns and adding Year to showcase forecasts in original scale

results.drop(['Beef_and_veal_diff', 'Poultry_diff', 'Sheep_diff'], axis=1, inplace= True)
results['Year'] = [2022, 2023, 2024, 2025]
results

,Pork,Beef_and_veal,Poultry,Sheep,Year
0,26670.304161,10450.671233,21583.143101,1234.735798,2022
1,26692.161697,10845.952097,21783.658412,1238.178847,2023
2,26710.165259,10700.402600,21982.155589,1218.505320,2024
3,26465.735271,10554.011435,22459.850265,1196.487596,2025


In [60]:
results['Year'] = pd.to_datetime(results['Year'], format = '%Y').dt.strftime('%Y') ## Changing Year to datetime type

In [61]:
results

,Pork,Beef_and_veal,Poultry,Sheep,Year
0,26670.304161,10450.671233,21583.143101,1234.735798,2022
1,26692.161697,10845.952097,21783.658412,1238.178847,2023
2,26710.165259,10700.402600,21982.155589,1218.505320,2024
3,26465.735271,10554.011435,22459.850265,1196.487596,2025


In [62]:
#To order the columns in the same way as required 
cols_to_order = ['Year','Pork'] #Columns to be placed at the beginning
new_columns = cols_to_order + (results.columns.drop(cols_to_order).tolist())

new_df = results[new_columns] #Create the ordered dataframe
new_df

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,2022,26670.304161,10450.671233,21583.143101,1234.735798
1,2023,26692.161697,10845.952097,21783.658412,1238.178847
2,2024,26710.165259,10700.402600,21982.155589,1218.505320
3,2025,26465.735271,10554.011435,22459.850265,1196.487596


In [63]:
meat = pd.concat([df_copy, results], ignore_index= True)
meat

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,1990,21638.200000,12415.740000,2054.650000,1809.260000
1,1991,21234.930000,12311.400000,8099.460000,1753.560000
2,1992,20991.890000,15833.920000,9777.320000,2051.710000
3,1993,24072.310000,15307.840000,9799.410000,2070.240000
4,1994,23715.500000,14543.260000,10474.860000,1981.300000
5,1995,23678.540000,13615.010000,11014.160000,1940.390000
6,1996,23315.000000,13149.180000,11370.330000,1894.990000
7,1997,23128.110000,12873.880000,11832.930000,1797.430000
8,1998,24218.650000,12574.530000,12144.540000,1786.360000
9,1999,24154.350000,11856.890000,11998.970000,1759.310000


In [64]:
meat = pd.merge(meat, original_pop, on = 'Year')  # Merging with original dataframe to have origial and forecasted data
meat.rename(columns={"Value":"Population"},inplace= True)
meat

,Year,Pork,Beef_and_veal,Poultry,Sheep,Population
0,1990,21638.200000,12415.740000,2054.650000,1809.260000,448805.382000
1,1991,21234.930000,12311.400000,8099.460000,1753.560000,450298.107000
2,1992,20991.890000,15833.920000,9777.320000,2051.710000,688358.946000
3,1993,24072.310000,15307.840000,9799.410000,2070.240000,705070.033000
4,1994,23715.500000,14543.260000,10474.860000,1981.300000,705611.100000
5,1995,23678.540000,13615.010000,11014.160000,1940.390000,705768.653000
6,1996,23315.000000,13149.180000,11370.330000,1894.990000,705842.588000
7,1997,23128.110000,12873.880000,11832.930000,1797.430000,705880.858000
8,1998,24218.650000,12574.530000,12144.540000,1786.360000,705845.172000
9,1999,24154.350000,11856.890000,11998.970000,1759.310000,705683.958000


In [65]:
meat.to_csv("Data/VARMAX_Forecast_with_Holt.csv") # Storing forecasts into a .csv file

## VAR Model Implementation to Forecast Meat Consumption of all types from 2022 through 2025

In [66]:
df = copy.deepcopy(df_copy)
df

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,1990,21638.20,12415.74,2054.65,1809.26
1,1991,21234.93,12311.40,8099.46,1753.56
2,1992,20991.89,15833.92,9777.32,2051.71
3,1993,24072.31,15307.84,9799.41,2070.24
4,1994,23715.50,14543.26,10474.86,1981.30
5,1995,23678.54,13615.01,11014.16,1940.39
6,1996,23315.00,13149.18,11370.33,1894.99
7,1997,23128.11,12873.88,11832.93,1797.43
8,1998,24218.65,12574.53,12144.54,1786.36
9,1999,24154.35,11856.89,11998.97,1759.31


#### Data Preprocessing

In [67]:
## Beef_and_veal, Poultry, and Sheep are non-stationary. Differencing to ensure data is stationary.

df['Beef_and_veal_diff'] = df['Beef_and_veal'].diff().dropna()
df['Poultry_diff'] = df['Poultry'].diff().dropna()
df['Sheep_diff'] = df['Sheep'].diff().dropna()

In [68]:
df.drop(df.loc[df['Year'] == '1990'].index, inplace= True) # Dropping the row corresponding to year 1990 as it is NaN after differencing

In [69]:
df.drop(['Beef_and_veal', 'Poultry', 'Sheep'], axis= 1, inplace= True)
df.set_index('Year', inplace=True)
df.head()

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
Year,,,,
1991,21234.93,-104.34,6044.81,-55.70
1992,20991.89,3522.52,1677.86,298.15
1993,24072.31,-526.08,22.09,18.53
1994,23715.50,-764.58,675.45,-88.94
1995,23678.54,-928.25,539.30,-40.91


In [70]:
# Splitting the data into train and test sets
n_obs = len(df)
train_size = int(0.8 * n_obs)
train, test = df[0:train_size], df[train_size:]

In [71]:
# Standardizing the data

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)

test_scaled = scaler.transform(test)

#### Data Modelling: VAR(p=1)

In [74]:
# Fitting the VAR model
model = VAR(train_scaled)
model_fitted = model.fit(1) 

# Print out the VAR results
print(model_fitted.summary())

# Forecasting
forecast = model_fitted.forecast(y=train_scaled[-model_fitted.k_ar:], steps=len(test))

print(forecast)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 02, Sep, 2023
Time:                     17:19:14
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -5.24481
Nobs:                     23.0000    HQIC:                  -5.98387
Log likelihood:          -38.8721    FPE:                 0.00202134
AIC:                     -6.23220    Det(Omega_mle):     0.000920277
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.142511         0.066429            2.145           0.032
L1.y1         0.740967         0.091848            8.067           0.000
L1.y2         0.607302         0.129552            4.688           0.000
L1.y3        -0.228572

In [76]:
# Convert test_scaled back to dataframe
test_scaled_df = pd.DataFrame(test_scaled, columns=df.columns)

# Convert forecasted values into DataFrame for easier handling
forecast_df = pd.DataFrame(forecast, index=df.index[-len(test_scaled):], columns=df.columns)
forecast_df

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
Year,,,,
2015,1.033555,0.020375,-0.249273,-0.030116
2016,0.986397,-0.089476,-0.253222,-0.162268
2017,0.923845,-0.089415,-0.246485,-0.171021
2018,0.878524,-0.093905,-0.245723,-0.168919
2019,0.841434,-0.098647,-0.244357,-0.169657
2020,0.810973,-0.101868,-0.243138,-0.169759
2021,0.786197,-0.104531,-0.242190,-0.169769


#### Model Evaluation

In [77]:
# Calculate evaluation metrics for each series
for col in df.columns[:]:
    print(f"Results for {col}")
    print("-------------------------")
    
    mae = mean_absolute_error(test_scaled_df[col], forecast_df[col])
    rmse = mean_squared_error(test_scaled_df[col], forecast_df[col], squared=False)
    mape = mean_absolute_percentage_error(test_scaled_df[col], forecast_df[col])
    
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}")
    print("\n")

Results for Pork
-------------------------
MAE: 0.5273322003370654
RMSE: 0.57297234005479
MAPE: 0.3575819015752558


Results for Beef_and_veal_diff
-------------------------
MAE: 0.1417610750201042
RMSE: 0.182508456074098
MAPE: 1.9135059710054294


Results for Poultry_diff
-------------------------
MAE: 0.2015381719981094
RMSE: 0.21380376247377655
MAPE: 1.3306565469277511


Results for Sheep_diff
-------------------------
MAE: 0.36240855418970247
RMSE: 0.45044777278901155
MAPE: 1.6053360168728843




#### Retraining the VAR(1) model using the entire dataset

In [78]:
print(df.tail())

          Pork  Beef_and_veal_diff  Poultry_diff  Sheep_diff
Year                                                        
2017  26705.10              -92.47        246.62      -38.32
2018  27007.08              201.90        541.69        7.08
2019  26366.19             -194.00         62.53      -21.98
2020  26060.52             -217.28         -9.40      -31.36
2021  26719.89              -41.16        130.04       10.73


In [79]:
## Standardizing the data

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [80]:
# Fitting the VAR model
model = VAR(df_scaled)
model_fitted = model.fit(1) 

# Print out the VAR results
print(model_fitted.summary())

# Forecasting
forecast = model_fitted.forecast(y=df_scaled[-model_fitted.k_ar:], steps=4)

print(forecast)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 02, Sep, 2023
Time:                     17:24:35
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -5.38823
Nobs:                     30.0000    HQIC:                  -6.02353
Log likelihood:          -55.4372    FPE:                 0.00181838
AIC:                     -6.32236    Det(Omega_mle):     0.000981518
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.116352         0.055709            2.089           0.037
L1.y1         0.810379         0.069308           11.692           0.000
L1.y2         0.461527         0.104081            4.434           0.000
L1.y3        -0.160712

In [81]:
forecast_rescaled = scaler.inverse_transform(forecast)
forecast_rescaled

array([[ 2.65850176e+04, -2.02294924e+02,  3.11553291e+02,
        -2.91247351e+01],
       [ 2.64327920e+04, -9.61368920e+01,  3.43494010e+02,
        -2.21142224e+01],
       [ 2.63773685e+04, -1.07286906e+02,  3.34372626e+02,
        -2.13250591e+01],
       [ 2.63208083e+04, -1.14041472e+02,  3.37838382e+02,
        -2.20725227e+01]])

In [82]:
actual_forecast_df = pd.DataFrame(forecast_rescaled, columns=df.columns)
actual_forecast_df

,Pork,Beef_and_veal_diff,Poultry_diff,Sheep_diff
0,26585.017579,-202.294924,311.553291,-29.124735
1,26432.791984,-96.136892,343.494010,-22.114222
2,26377.368549,-107.286906,334.372626,-21.325059
3,26320.808348,-114.041472,337.838382,-22.072523


In [83]:
# Function for Reverse Differencing

def reverse_differencing(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
        # Roll back 1st Diff
        # df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
        df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
    return df_fc

In [84]:
last_obs_beef_and_veal = float(original_beef_and_veal.iloc[-1:])
last_obs_poultry = float(original_poultry.iloc[-1:])
last_obs_sheep = float(original_sheep.iloc[-1:])

In [85]:
df_train = df_copy.drop(['Year', 'Pork'], axis = 1)
print(df_train.tail())

forecasted_df = actual_forecast_df.drop(['Pork'], axis = 1)
print(forecasted_df)

    Beef_and_veal   Poultry    Sheep
27       10810.73  19935.89  1338.59
28       11012.63  20477.58  1345.67
29       10818.63  20540.11  1323.69
30       10601.35  20530.71  1292.33
31       10560.19  20660.75  1303.06
   Beef_and_veal_diff  Poultry_diff  Sheep_diff
0         -202.294924    311.553291  -29.124735
1          -96.136892    343.494010  -22.114222
2         -107.286906    334.372626  -21.325059
3         -114.041472    337.838382  -22.072523


In [86]:
df_results = reverse_differencing(df_train, forecasted_df, second_diff = False)
print(df_results)

   Beef_and_veal_diff  Poultry_diff  Sheep_diff  Beef_and_veal       Poultry  \
0         -202.294924    311.553291  -29.124735   10357.895076  20972.303291   
1          -96.136892    343.494010  -22.114222   10261.758184  21315.797300   
2         -107.286906    334.372626  -21.325059   10154.471278  21650.169926   
3         -114.041472    337.838382  -22.072523   10040.429806  21988.008308   

         Sheep  
0  1273.935265  
1  1251.821043  
2  1230.495983  
3  1208.423461  


In [87]:
results = pd.concat([actual_forecast_df, df_results], axis = 1) # Concatenating to get all forecasted series

In [88]:
# Dropping differenced columns and adding Year to showcase forecasts in original scale

results.drop(['Beef_and_veal_diff', 'Poultry_diff', 'Sheep_diff'], axis=1, inplace= True)
results['Year'] = [2022, 2023, 2024, 2025]
results

,Pork,Beef_and_veal,Poultry,Sheep,Year
0,26585.017579,10357.895076,20972.303291,1273.935265,2022
1,26432.791984,10261.758184,21315.797300,1251.821043,2023
2,26377.368549,10154.471278,21650.169926,1230.495983,2024
3,26320.808348,10040.429806,21988.008308,1208.423461,2025


In [89]:
results['Year'] = pd.to_datetime(results['Year'], format = '%Y').dt.strftime('%Y') ## Changing Year to datetime type
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pork           4 non-null      float64
 1   Beef_and_veal  4 non-null      float64
 2   Poultry        4 non-null      float64
 3   Sheep          4 non-null      float64
 4   Year           4 non-null      object 
dtypes: float64(4), object(1)
memory usage: 288.0+ bytes


In [90]:
#To order the columns in the same way as required 
cols_to_order = ['Year','Pork'] #Columns to be placed at the beginning


#To avoid writing everything again and rearrange columns more efficiently
new_columns = cols_to_order + (results.columns.drop(cols_to_order).tolist())

new_df = results[new_columns] #Create the ordered dataframe
new_df

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,2022,26585.017579,10357.895076,20972.303291,1273.935265
1,2023,26432.791984,10261.758184,21315.797300,1251.821043
2,2024,26377.368549,10154.471278,21650.169926,1230.495983
3,2025,26320.808348,10040.429806,21988.008308,1208.423461


In [91]:
meat = pd.concat([df_copy, results], ignore_index= True) # Concatenating with original dataframe to have origial and forecasted data
meat

,Year,Pork,Beef_and_veal,Poultry,Sheep
0,1990,21638.200000,12415.740000,2054.650000,1809.260000
1,1991,21234.930000,12311.400000,8099.460000,1753.560000
2,1992,20991.890000,15833.920000,9777.320000,2051.710000
3,1993,24072.310000,15307.840000,9799.410000,2070.240000
4,1994,23715.500000,14543.260000,10474.860000,1981.300000
5,1995,23678.540000,13615.010000,11014.160000,1940.390000
6,1996,23315.000000,13149.180000,11370.330000,1894.990000
7,1997,23128.110000,12873.880000,11832.930000,1797.430000
8,1998,24218.650000,12574.530000,12144.540000,1786.360000
9,1999,24154.350000,11856.890000,11998.970000,1759.310000


In [92]:
meat.to_csv("Data/VAR_Forecast_with_Holt.csv") # Storing the data in a .csv file